In [1]:
import tensorflow as tf
import numpy as np

In [3]:
'''In this step we download and load the data set and split it into two parts ( Train and test sets ) then printing the shape 
of each set. The way we will split is :  90:5:5 ratio (train:validation:test)
'''
#  Because we have a set of text files we will use tf.keras.preprocessing.text_dataset_from_directory to generate a labeled tf.data.Dataset
batch_size = 32
#Here we are generating train set from train directory, the size is 80%
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
#Here we are generating validation set from train directory, the size is 20%
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
#Here we are generating test set from test directory
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
#Finally we will visualize the number of exemples in each created set
print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


In [5]:
# It's important to take a look at your raw data to ensure your normalization
# and tokenization will work as expected. We can do that by taking a few
# examples from the training set and looking at them.
# This is one of the places where eager execution shines:
# we can just evaluate these tensors using .numpy()
# instead of needing to evaluate them in a Session/Graph context.
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b'I am very disappointed with "K-911." The original "good" quality of "K-9" doesn\'t exist any more. This is more like a sitcom! Some of casts from original movie returned and got some of my memory back. The captain of Dooley now loves to hit him like a scene from old comedy show. That was crazy. What\'s the deal with the change of Police? It seems like they are now LAPD! Not San Diego PD. It is a completely different movie from "'
0
b"Giallo fans, seek out this rare film. It is well written, and full of all sorts of the usual low lifes that populate these films. I don't want to give anything away, so I wont even say anything about the plot. The whole movie creates a very bizarre atmosphere, and you don't know what to expect or who to suspect. Recommended! The only place I've seen to get this film in english is from European Trash Cinema, for $15."
1
b"Terry Gilliam's and David Peoples' teamed up to create one of the most intelligent and creative science fiction movies of the '90's. Pe

In [6]:
from tensorflow.keras.layers import TextVectorization
import string
import re

#We will remove the HTML break tags "<br /> from the text", and we will create a custom standardization function
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )


# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

# To normalize, split, and map strings to integers we need to add a vectorization layer for texts
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)


# Let's make a text-only dataset (no labels):
text_ds = raw_train_ds.map(lambda x, y: x)
# Let's call `adapt`:
vectorize_layer.adapt(text_ds)

# Vectorize the data

In [7]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

# Build model


In [8]:
from tensorflow.keras import layers
'''
Now we will build a simple 1D convnet starting with an Embedding layer.
Before fitting our model, we choosed ReLU as loss function from hiddel layer, and sigmoid activation function for output layer.
'''
# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train model

In [9]:
epochs = 3

# Fit the model using the train and test datasets.
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3
625/625 [==============================] - 71s 109ms/step - loss: 0.4820 - accuracy: 0.7358 - val_loss: 0.3154 - val_accuracy: 0.8656
Epoch 2/3
625/625 [==============================] - 56s 90ms/step - loss: 0.2160 - accuracy: 0.9161 - val_loss: 0.3182 - val_accuracy: 0.8762
Epoch 3/3
625/625 [==============================] - 57s 91ms/step - loss: 0.1146 - accuracy: 0.9579 - val_loss: 0.3888 - val_accuracy: 0.8682


# Test model

In [10]:
model.evaluate(test_ds)

782/782 [==============================] - 137s 172ms/step - loss: 0.3998 - accuracy: 0.8601


[0.3997553884983063, 0.8600800037384033]

In [11]:
'''To create model that can processe raw strings, we can use the weight we trained'''
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)

782/782 [==============================] - 14s 16ms/step - loss: 0.3998 - accuracy: 0.8601


[0.3997551202774048, 0.8600800037384033]

In this example we have build and trained a text classification model starting from raw text. We remove the HTML space tags andc create our standardization function, and We create a vectorization layer for word splitting & indexing. Finally we build a model using the weight we trained that capable of preprocessing raw strings.